### Analysis of Cyclistic Data in the Past 12 Months
Visualizations are done in Tableau after the dataframe is cut here for each visualization purpose

In [1]:
import pandas as pd
df = pd.read_csv("cyclistic_2021.csv")

In [2]:
df.head()

,Unnamed: 0,ride_id,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day
0,0,BD0A6FF6FFF9B921,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:40,Dearborn St & Erie St,St. Clair St & Erie St,casual,0 days 00:09:40,Sunday
1,1,96A7A7A4BDE4F82D,electric_bike,2020-11-01 10:03:26,2020-11-01 10:14:45,Franklin St & Illinois St,Noble St & Milwaukee Ave,casual,0 days 00:11:19,Sunday
2,2,C61526D06582BDC5,electric_bike,2020-11-01 00:34:05,2020-11-01 01:03:06,Lake Shore Dr & Monroe St,Federal St & Polk St,casual,0 days 00:29:01,Sunday
3,3,E533E89C32080B9E,electric_bike,2020-11-01 00:45:16,2020-11-01 00:54:31,Leavitt St & Chicago Ave,Stave St & Armitage Ave,casual,0 days 00:09:15,Sunday
4,4,1C9F4EF18C168C60,electric_bike,2020-11-01 15:43:25,2020-11-01 16:16:52,Buckingham Fountain,Buckingham Fountain,casual,0 days 00:33:27,Sunday


### Basic Analysis

In [3]:
dropp = df.columns[:2]
df = df.drop(columns=dropp)

In [4]:
df["ride_length"] = pd.to_timedelta(df["ride_length"])

In [5]:
df.dtypes

rideable_type                  object
started_at                     object
ended_at                       object
start_station_name             object
end_station_name               object
member_casual                  object
ride_length           timedelta64[ns]
day                            object
dtype: object

We'll use dfa from now on.

In [6]:
dfa = df

In [7]:
df.rideable_type.unique()

array(['electric_bike', 'docked_bike', 'classic_bike'], dtype=object)

How many rides in the past 12 months?

In [35]:
len(dfa)

5607239

This is a huge dataset, we'll have to split the data for each analysis:
#### Time-centric:
- Which days have the most riders? (split(subscription), stack(bike_type))
- Does ride_length have anything to do with subscription | bike_type | weekday
- Busy hours?

#### Location-based:
- What are the most common routes?

#### Mixed:
- What are the most common routes during rush hour?

#### Usage

### Daily Riders by Bike-Type and subscription

In [10]:
drop1 = dfa.columns[1:5]
tc = dfa.drop(columns=drop1)
tc.head()

,rideable_type,member_casual,ride_length,day
0,electric_bike,casual,0 days 00:09:40,Sunday
1,electric_bike,casual,0 days 00:11:19,Sunday
2,electric_bike,casual,0 days 00:29:01,Sunday
3,electric_bike,casual,0 days 00:09:15,Sunday
4,electric_bike,casual,0 days 00:33:27,Sunday


In [11]:
days = tc.groupby(["day", "member_casual", "rideable_type"]).count().reset_index()
tca = days.pivot(index=['day', 'member_casual'], columns='rideable_type', values="ride_length")
tca

rideable_type            classic_bike  docked_bike  electric_bike
day       member_casual                                          
Friday    casual               173335        48029         143167
          member               268662        15941         158348
Monday    casual               133746        41319         114460
          member               259995        18082         144459
Saturday  casual               297789        89544         180539
          member               277335        15579         146239
Sunday    casual               254912        78589         155024
          member               245449        13753         124055
Thursday  casual               133990        31837         117989
          member               272251        14552         153554
Tuesday   casual               124291        33169         118974
          member               288275        13652         163616
Wednesday casual               128788        30940         116935
          member               291575        14011         160490

### Ride Length Correlation

In [12]:
dfa.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day
0,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:40,Dearborn St & Erie St,St. Clair St & Erie St,casual,0 days 00:09:40,Sunday
1,electric_bike,2020-11-01 10:03:26,2020-11-01 10:14:45,Franklin St & Illinois St,Noble St & Milwaukee Ave,casual,0 days 00:11:19,Sunday
2,electric_bike,2020-11-01 00:34:05,2020-11-01 01:03:06,Lake Shore Dr & Monroe St,Federal St & Polk St,casual,0 days 00:29:01,Sunday
3,electric_bike,2020-11-01 00:45:16,2020-11-01 00:54:31,Leavitt St & Chicago Ave,Stave St & Armitage Ave,casual,0 days 00:09:15,Sunday
4,electric_bike,2020-11-01 15:43:25,2020-11-01 16:16:52,Buckingham Fountain,Buckingham Fountain,casual,0 days 00:33:27,Sunday


In [13]:
rlc = dfa.drop(columns=["started_at", "ended_at"])
rlc["route"] = rlc["start_station_name"] + " - " + rlc["end_station_name"]
rlc = rlc.drop(columns=["start_station_name", "end_station_name"])
rlc.head()

,rideable_type,member_casual,ride_length,day,route
0,electric_bike,casual,0 days 00:09:40,Sunday,Dearborn St & Erie St - St. Clair St & Erie St
1,electric_bike,casual,0 days 00:11:19,Sunday,Franklin St & Illinois St - Noble St & Milwauk...
2,electric_bike,casual,0 days 00:29:01,Sunday,Lake Shore Dr & Monroe St - Federal St & Polk St
3,electric_bike,casual,0 days 00:09:15,Sunday,Leavitt St & Chicago Ave - Stave St & Armitage...
4,electric_bike,casual,0 days 00:33:27,Sunday,Buckingham Fountain - Buckingham Fountain


In [14]:
# rlc.to_csv("rlc.csv")

Route averages

In [15]:
dropra = rlc.columns[:2]
ra = rlc.drop(columns=dropra)
ra.head()

,ride_length,day,route
0,0 days 00:09:40,Sunday,Dearborn St & Erie St - St. Clair St & Erie St
1,0 days 00:11:19,Sunday,Franklin St & Illinois St - Noble St & Milwauk...
2,0 days 00:29:01,Sunday,Lake Shore Dr & Monroe St - Federal St & Polk St
3,0 days 00:09:15,Sunday,Leavitt St & Chicago Ave - Stave St & Armitage...
4,0 days 00:33:27,Sunday,Buckingham Fountain - Buckingham Fountain


In [16]:
import numpy as np
ra.ride_length = ra.ride_length.values.astype(np.int64)

In [17]:
tb1 = ra.groupby("day").ride_length.mean().reset_index()
# tb1["ride_length"] = pd.to_timedelta(tb1["ride_length"])
tb1

,day,ride_length
0,Friday,1.274523e+12
1,Monday,1.253097e+12
2,Saturday,1.587158e+12
3,Sunday,1.683962e+12
4,Thursday,1.129433e+12
5,Tuesday,1.112848e+12
6,Wednesday,1.109119e+12


In [18]:
# tb1.to_csv("ride_length_avg.csv")

In [19]:
tb2 = ra.groupby("route").ride_length.mean().reset_index()
tb2["ride_length"] = pd.to_timedelta(tb2["ride_length"])
tb2 = tb2.sort_values("ride_length", ascending=False).head(10)
tb2

,route,ride_length
79755,Lake Shore Dr & Ohio St - Base - 2132 W Hubbar...,32 days 15:30:51
53055,Dusable Harbor - Warren Park East,28 days 06:44:36
20316,Canal St & Adams St - Stony Island Ave & 82nd St,27 days 09:19:09
135828,Throop St & 52nd St - Base - 2132 W Hubbard Wa...,27 days 00:42:55
70379,Indiana Ave & Roosevelt Rd - Greenwood Ave & 7...,26 days 20:45:16
37874,Clyde Ave & 87th St - MLK Jr Dr & 63rd St,24 days 05:34:00
59429,Franklin St & Chicago Ave - Homewood Ave & 115...,22 days 06:38:51
59325,Franklin St & Chicago Ave - Cicero Ave & Lake St,22 days 02:47:00
79129,Lake Shore Dr & Monroe St - Kedzie Ave & Roose...,22 days 00:02:44
133850,Stockton Dr & Wrightwood Ave - Base - 2132 W H...,21 days 14:46:00


This seems to be irrelevant.

### Busy Hours

In [20]:
dfa.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'end_station_name', 'member_casual', 'ride_length', 'day'],
      dtype='object')

In [21]:
dfa["started_at"] = pd.to_datetime(dfa["started_at"])
drop3 = dfa.columns[2:5]
bh = dfa.drop(columns=drop3)
bh["hour"] = bh["started_at"].dt.hour
bh.head()

,rideable_type,started_at,member_casual,ride_length,day,hour
0,electric_bike,2020-11-01 13:36:00,casual,0 days 00:09:40,Sunday,13
1,electric_bike,2020-11-01 10:03:26,casual,0 days 00:11:19,Sunday,10
2,electric_bike,2020-11-01 00:34:05,casual,0 days 00:29:01,Sunday,0
3,electric_bike,2020-11-01 00:45:16,casual,0 days 00:09:15,Sunday,0
4,electric_bike,2020-11-01 15:43:25,casual,0 days 00:33:27,Sunday,15


In [22]:
tcc = bh.groupby(["hour", "day", "member_casual"]).count().reset_index().drop(columns=["started_at", "ride_length"])
tccd = tcc.pivot(index=['hour', 'member_casual'], columns='day', values="rideable_type")
tccd

day                 Friday  Monday  Saturday  Sunday  Thursday  Tuesday  \
hour member_casual                                                        
0    casual           6054    4892     13963   18268      3861     3125   
     member           4158    2658      8547    9408      2979     2046   
1    casual           3968    3214     12285   13798      2222     1852   
     member           2306    1571      6683    7164      1411     1020   
2    casual           2141    2051      7892    9597      1327     1090   
     member           1214     848      3956    4222       684      621   
3    casual           1270    1354      3751    5268       808      707   
     member            712     616      1943    2375       506      447   
4    casual           1132    1408      2016    2946       819      752   
     member           1038    1071      1262    1570       982     1027   
5    casual           1622    1668      1804    2359      1630     1717   
     member           4542    4866      1995    1820      4978     5804   
6    casual           3935    3802      2804    2917      4005     4244   
     member          12819   13635      4814    3950     14228    16466   
7    casual           6908    6888      4959    4735      7253     7927   
     member          22380   24559      8838    6544     26360    30260   
8    casual           8752    8684      9904    8244      8957     9315   
     member          25245   26755     14871   10695     28677    32251   
9    casual           9583    8781     18084   15443      7850     8456   
     member          17954   16884     21929   16622     18186    19223   
10   casual          12846   11258     28759   25071      9451     9574   
     member          16703   15021     28399   23186     15106    15351   
11   casual          16696   14868     37225   33533     11785    11762   
     member          20573   18988     32359   28058     18490    18749   
12   casual          21089   18236     43921   38469     13847    14839   
     member          24718   22525     34557   31202     22474    23249   
13   casual          22756   19269     47882   41670     14566    15720   
     member          25107   22155     34530   31586     20892    22868   
14   casual          23996   19833     48102   42298     15531    16555   
     member          25543   21541     34313   31470     20405    21657   
15   casual          27103   21216     46414   43513     17972    18665   
     member          30357   26381     32982   32397     25679    27835   
16   casual          31001   24559     44203   40344     22703    23197   
     member          39469   37402     31791   31300     37590    41172   
17   casual          36441   29659     40563   35963     30956    32332   
     member          45879   50415     30580   28923     50653    57541   
18   casual          34018   26706     36613   29196     29816    28730   
     member          39251   41896     27505   24927     43630    46495   
19   casual          26066   20068     29765   23087     23143    21597   
     member          28020   29086     22544   19562     30611    31531   
20   casual          18924   14342     23321   17517     16965    14918   
     member          18682   18729     16788   13875     20597    20217   
21   casual          16474   11469     20830   14021     14828    11984   
     member          13874   12458     13807   10186     15426    14220   
22   casual          15994    9522     21887   12213     13765    10850   
     member          11829    8094     12665    7613     12142     9915   
23   casual          15762    5778     20925    8055      9756     6526   
     member          10578    4382     11495    4602      7671     5578   

day                 Wednesday  
hour member_casual             
0    casual              3428  
     member              2580  
1    casual              1680  
     member              1203  
2    casual              1109  
     member              

In [23]:
tccs = bh.groupby(["hour", "member_casual"]).count().reset_index().drop(columns=["day","started_at","ride_length"])
tccs = tccs.pivot(index='hour', columns='member_casual', values="rideable_type")
tccs

member_casual,casual,member
hour,,
0,53591,32376
1,39019,21358
2,25207,12196
3,13897,7010
4,9812,7979
5,12383,29467
6,25725,81676
7,46563,148318
8,63416,170365


### Location-Based

In [24]:
dfa.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'end_station_name', 'member_casual', 'ride_length', 'day'],
      dtype='object')

In [46]:
lba = dfa
lba["hour"] = lba["started_at"].dt.hour
lb = lba.drop(columns=["started_at", "ended_at"])
lb.head()

,rideable_type,start_station_name,end_station_name,member_casual,ride_length,day,hour
0,electric_bike,Dearborn St & Erie St,St. Clair St & Erie St,casual,0 days 00:09:40,Sunday,13
1,electric_bike,Franklin St & Illinois St,Noble St & Milwaukee Ave,casual,0 days 00:11:19,Sunday,10
2,electric_bike,Lake Shore Dr & Monroe St,Federal St & Polk St,casual,0 days 00:29:01,Sunday,0
3,electric_bike,Leavitt St & Chicago Ave,Stave St & Armitage Ave,casual,0 days 00:09:15,Sunday,0
4,electric_bike,Buckingham Fountain,Buckingham Fountain,casual,0 days 00:33:27,Sunday,15


In [47]:
lb["route"] = lb["start_station_name"] + " - " + lb["end_station_name"]
lb.head()

,rideable_type,start_station_name,end_station_name,member_casual,ride_length,day,hour,route
0,electric_bike,Dearborn St & Erie St,St. Clair St & Erie St,casual,0 days 00:09:40,Sunday,13,Dearborn St & Erie St - St. Clair St & Erie St
1,electric_bike,Franklin St & Illinois St,Noble St & Milwaukee Ave,casual,0 days 00:11:19,Sunday,10,Franklin St & Illinois St - Noble St & Milwauk...
2,electric_bike,Lake Shore Dr & Monroe St,Federal St & Polk St,casual,0 days 00:29:01,Sunday,0,Lake Shore Dr & Monroe St - Federal St & Polk St
3,electric_bike,Leavitt St & Chicago Ave,Stave St & Armitage Ave,casual,0 days 00:09:15,Sunday,0,Leavitt St & Chicago Ave - Stave St & Armitage...
4,electric_bike,Buckingham Fountain,Buckingham Fountain,casual,0 days 00:33:27,Sunday,15,Buckingham Fountain - Buckingham Fountain


#### Route Analysis
Let's see which routes members and casual riders usually take most by listing the top 10 most common routes by causal riders and compare that to those taken by members

In [48]:
lba= lb.groupby(["route", "member_casual"]).day.count().reset_index()
lba = lba.sort_values("day", ascending=False)
lbac = lba[lba["member_casual"]=="casual"].head(10)
lbac

,route,member_casual,day
218869,Streeter Dr & Grand Ave - Streeter Dr & Grand Ave,casual,11790
160534,Millennium Park - Millennium Park,casual,6303
156891,Michigan Ave & Oak St - Michigan Ave & Oak St,casual,6043
127539,Lake Shore Dr & Monroe St - Lake Shore Dr & Mo...,casual,5010
21445,Buckingham Fountain - Buckingham Fountain,casual,3567
220087,Theater on the Lake - Theater on the Lake,casual,3344
218664,Streeter Dr & Grand Ave - Millennium Park,casual,3286
163715,Montrose Harbor - Montrose Harbor,casual,3112
113912,Indiana Ave & Roosevelt Rd - Indiana Ave & Roo...,casual,2988
194443,Shedd Aquarium - Shedd Aquarium,casual,2956


These 10 routes could be where we advertise on, seeing most of the route-takers are casual riders

In [49]:
lbam = lba[lba["member_casual"]=="member"].head(10)
lbam

,route,member_casual,day
88520,Ellis Ave & 60th St - Ellis Ave & 55th St,member,4142
88094,Ellis Ave & 55th St - Ellis Ave & 60th St,member,3735
88643,Ellis Ave & 60th St - University Ave & 57th St,member,2896
221719,University Ave & 57th St - Ellis Ave & 60th St,member,2818
211417,State St & 33rd St - Calumet Ave & 33rd St,member,1894
29899,Calumet Ave & 33rd St - State St & 33rd St,member,1873
146945,Loomis St & Lexington St - Morgan St & Polk St,member,1848
166258,Morgan St & Polk St - Loomis St & Lexington St,member,1635
147395,MLK Jr Dr & 29th St - State St & 33rd St,member,1458
211559,State St & 33rd St - MLK Jr Dr & 29th St,member,1450


They are different routes... We'll have to further analye the casuals routes

In [50]:
top3= ("Streeter Dr & Grand Ave - Streeter Dr & Grand Ave", 
                    "Millennium Park - Millennium Park", 
                     "Michigan Ave & Oak St - Michigan Ave & Oak St")
cr = lb[lb["route"].isin(top3)]
cr.head()

,rideable_type,start_station_name,end_station_name,member_casual,ride_length,day,hour,route
1669,electric_bike,Michigan Ave & Oak St,Michigan Ave & Oak St,casual,0 days 00:50:58,Friday,8,Michigan Ave & Oak St - Michigan Ave & Oak St
1895,electric_bike,Millennium Park,Millennium Park,casual,0 days 00:24:31,Wednesday,20,Millennium Park - Millennium Park
1896,electric_bike,Millennium Park,Millennium Park,casual,0 days 01:38:31,Thursday,0,Millennium Park - Millennium Park
3102,electric_bike,Millennium Park,Millennium Park,casual,0 days 00:20:48,Sunday,21,Millennium Park - Millennium Park
3137,electric_bike,Michigan Ave & Oak St,Michigan Ave & Oak St,casual,0 days 01:03:41,Saturday,13,Michigan Ave & Oak St - Michigan Ave & Oak St


In [53]:
cra = cr.groupby(["route", "day", "hour"]).ride_length.count().reset_index()
cra.pivot(index=["route", "day"], columns="hour", values="ride_length")

hour                                                           0     1     2   \
route                                             day                           
Michigan Ave & Oak St - Michigan Ave & Oak St     Friday      NaN   2.0   4.0   
                                                  Monday      9.0   6.0   1.0   
                                                  Saturday   11.0   3.0  22.0   
                                                  Sunday     18.0  20.0   6.0   
                                                  Thursday   10.0   2.0   4.0   
                                                  Tuesday     4.0  12.0   NaN   
                                                  Wednesday  16.0   8.0   4.0   
Millennium Park - Millennium Park                 Friday      9.0  11.0  14.0   
                                                  Monday     30.0  14.0  12.0   
                                                  Saturday   37.0  12.0  17.0   
                                                  Sunday     84.0  36.0  36.0   
                                                  Thursday   17.0   9.0   2.0   
                                                  Tuesday    11.0  10.0   2.0   
                                                  Wednesday  20.0   NaN   3.0   
Streeter Dr & Grand Ave - Streeter Dr & Grand Ave Friday      1.0   4.0   NaN   
                                                  Monday      7.0   6.0   6.0   
                                                  Saturday   30.0   7.0   6.0   
                                                  Sunday     32.0  26.0   9.0   
                                                  Thursday    5.0   1.0   5.0   
                                                  Tuesday     6.0   NaN   NaN   
                                                  Wednesday  21.0   NaN   NaN   

hour                                                           3    4     5   \
route                                             day                          
Michigan Ave & Oak St - Michigan Ave & Oak St     Friday      2.0  NaN   NaN   
                                                  Monday      1.0  NaN   3.0   
                                                  Saturday    1.0  1.0   2.0   
                                                  Sunday      7.0  NaN   4.0   
                                                  Thursday    NaN  NaN   NaN   
                                                  Tuesday     NaN  NaN   3.0   
                                                  Wednesday   3.0  NaN   3.0   
Millennium Park - Millennium Park                 Friday      5.0  NaN   NaN   
                                                  Monday      4.0  3.0   2.0   
                                                  Saturday   12.0  NaN   NaN   
                                                  Sunday     25.0  9.0   NaN   
                                                  Thursday    7.0  4.0   NaN   
                                                  Tuesday     1.0  6.0   2.0   
                                                  Wednesday   6.0  2.0   1.0   
Streeter Dr & Grand Ave - Streeter Dr & Grand Ave Friday      2.0  2.0   NaN   
                                                  Monday      1.0  1.0   3.0   
                                                  Saturday    NaN  4.0   5.0   
                                                  Sunday      4.0  4.0  12.0   
                                                  Thursday    NaN  NaN   NaN   
                                                  Tuesday     NaN  NaN   2.0   
                                                  Wednesday   1.0  1.0   4.0   

hour                                                           6     7     8   \
route                                             day                           
Michigan Ave & Oak St - Michigan Ave & Oak St     Friday      9.0  19.0  12.0   
                                                  Mo

In [54]:
# cra.to_csv("cra.csv")

### Usage

In [28]:
dfa.head()

,rideable_type,started_at,ended_at,start_station_name,end_station_name,member_casual,ride_length,day
0,electric_bike,2020-11-01 13:36:00,2020-11-01 13:45:40,Dearborn St & Erie St,St. Clair St & Erie St,casual,0 days 00:09:40,Sunday
1,electric_bike,2020-11-01 10:03:26,2020-11-01 10:14:45,Franklin St & Illinois St,Noble St & Milwaukee Ave,casual,0 days 00:11:19,Sunday
2,electric_bike,2020-11-01 00:34:05,2020-11-01 01:03:06,Lake Shore Dr & Monroe St,Federal St & Polk St,casual,0 days 00:29:01,Sunday
3,electric_bike,2020-11-01 00:45:16,2020-11-01 00:54:31,Leavitt St & Chicago Ave,Stave St & Armitage Ave,casual,0 days 00:09:15,Sunday
4,electric_bike,2020-11-01 15:43:25,2020-11-01 16:16:52,Buckingham Fountain,Buckingham Fountain,casual,0 days 00:33:27,Sunday


In [29]:
dfa.columns

Index(['rideable_type', 'started_at', 'ended_at', 'start_station_name',
       'end_station_name', 'member_casual', 'ride_length', 'day'],
      dtype='object')

In [30]:
g = dfa.drop(columns=['rideable_type', 'ended_at', 'start_station_name','end_station_name','ride_length', 'day'])
g["month"] = g["started_at"].dt.month_name()
g["year"] = g["started_at"].dt.year
g = g[g["year"]!=2020]
g.head()

,started_at,member_casual,month,year
259716,2021-01-23 16:14:19,member,January,2021
259717,2021-01-27 18:43:08,member,January,2021
259718,2021-01-21 22:35:54,member,January,2021
259719,2021-01-07 13:31:13,member,January,2021
259720,2021-01-23 02:24:02,casual,January,2021


In [31]:
growth = g.groupby(["month", "member_casual"]).count().reset_index()
growth.pivot(index="month", columns="member_casual", values="started_at")

member_casual,casual,member
month,,
April,136601,200629
August,412671,391681
February,10131,39491
January,18117,78717
July,442056,380354
June,370681,358914
March,84033,144463
May,256916,274717
November,106929,253049


In [32]:
g.to_csv("monthly.csv")